In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.types import StructType, StructField, DoubleType

In [2]:
df = pd.read_csv('data/output.csv')

In [3]:
df.head(1)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
0,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000.0,2016-02-26 13:38:27,52375.0,successful,224.0,US,52375.0


In [4]:
df.isnull().sum()

ID               0
name             3
category         0
main_category    0
currency         0
deadline         0
goal             0
launched         0
pledged          0
state            0
backers          0
country          0
usd pledged      0
dtype: int64

In [5]:
spark=SparkSession.builder\
    .master("local[*]")\
    .appName("NaiveBayes")\
    .getOrCreate()
sc=spark.sparkContext

23/05/08 14:48:00 WARN Utils: Your hostname, jimbo-G5-5587 resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlp0s20f3)
23/05/08 14:48:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/08 14:48:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Prepare data for training

# Drop unnecessary columns
df.drop(['ID', 'name', 'usd pledged'], axis=1, inplace=True)

# Drop rows with missing values
df.dropna(inplace=True)

# Calculate the project duration in days
df['deadline'] = pd.to_datetime(df['deadline'])
df['launched'] = pd.to_datetime(df['launched'])
df['duration'] = (df['deadline'] - df['launched']).dt.days
df.drop(['deadline', 'launched'], axis=1, inplace=True)

# convert all strings to lowercase
df['category'] = df['category'].str.lower()
df['main_category'] = df['main_category'].str.lower()
df['currency'] = df['currency'].str.lower()
df.drop(['category'], axis=1, inplace=True)

# Change the state column to 0 and 1
df["state"] = df["state"].map({"failed": 0, "successful": 1})

In [7]:
df.head(1)

,main_category,currency,goal,pledged,state,backers,country,duration
0,food,usd,50000.0,52375.0,1,224.0,US,35


In [8]:
df = pd.get_dummies(df, columns=["main_category", "currency", "country"])

In [9]:
df.columns

Index(['goal', 'pledged', 'state', 'backers', 'duration', 'main_category_art',
       'main_category_comics', 'main_category_crafts', 'main_category_dance',
       'main_category_design', 'main_category_fashion',
       'main_category_film & video', 'main_category_food',
       'main_category_games', 'main_category_journalism',
       'main_category_music', 'main_category_photography',
       'main_category_publishing', 'main_category_technology',
       'main_category_theater', 'currency_aud', 'currency_cad', 'currency_chf',
       'currency_dkk', 'currency_eur', 'currency_gbp', 'currency_hkd',
       'currency_jpy', 'currency_mxn', 'currency_nok', 'currency_nzd',
       'currency_sek', 'currency_sgd', 'currency_usd', 'country_AT',
       'country_AU', 'country_BE', 'country_CA', 'country_CH', 'country_DE',
       'country_DK', 'country_ES', 'country_FR', 'country_GB', 'country_HK',
       'country_IE', 'country_IT', 'country_JP', 'country_LU', 'country_MX',
       'country_NL', 'coun

In [10]:
df.head(6)

,goal,pledged,state,backers,duration,main_category_art,main_category_comics,main_category_crafts,main_category_dance,main_category_design,...,country_JP,country_LU,country_MX,country_NL,country_NO,country_NZ,country_None,country_SE,country_SG,country_US
0,50000.0,52375.00,1,224.0,35,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1000.0,1205.00,1,16.0,20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,12500.0,12700.00,1,100.0,30,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,250.0,250.00,1,7.0,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,20000.0,34268.00,1,624.0,35,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,175.0,701.66,1,66.0,20,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
# Define the map function
def map_function(line):
    # Extract the features and target from the line
    X = line[:-1]
    y = int(line[-1])

    # Create a list of tuples to store the feature index, value, target, and count
    tuples = []

    # Loop through each feature index and value
    for i, x in enumerate(X):
        # Append a tuple of feature index, value, target, and count to the list
        tuples.append(((i, x, y), 1))

    # Return the list of tuples
    return tuples

# Define the reduce function
def reduce_function(a, b):
    # Sum the counts for each tuple
    return a + b

In [12]:
spark_df = spark.createDataFrame(df)

In [13]:
train, test = spark_df.randomSplit([0.7, 0.3], seed=12345)

In [14]:
train = train.rdd
test = test.rdd

In [15]:
# Apply the map function to the data RDD
mapped_rdd = train.flatMap(map_function)

In [16]:
# Apply the reduce function to count the occurrences of each tuple
reduced_rdd = mapped_rdd.reduceByKey(reduce_function)

In [17]:
# Collect the frequency table as a dictionary
frequency_table = reduced_rdd.collectAsMap()

23/05/08 14:50:21 WARN TaskSetManager: Stage 0 contains a task of very large size (3734 KiB). The maximum recommended task size is 1000 KiB.


In [22]:
def predict(line, frequency_table):
    frequency_table = frequency_table.value

    # Extract the features and target from the line
    X = line[:-1]
    y = int(line[-1])

    # Initialize an empty list to store the posterior probabilities for each class label
    posteriors = []

    # Loop through each class label (0 or 1)
    for label in [0, 1]:
        # Initialize the posterior probability as the prior probability of the class label with Laplace smoothing
        posterior = (frequency_table[(-1, label)] + 1) / (frequency_table[(-1, -1)] + 2)

        # Loop through each feature index and value
        for i, x in enumerate(X):
            x = float(x)
            try:
                posterior *= (frequency_table[(i, x, label)] + 1) / (frequency_table[(-1, label)] + len(frequency_table))
            except:
                #print("Error: Feature %d value %s not in training data" % (i, x))
                posterior *= (0 + 1) / (frequency_table[(-1, label)] + len(frequency_table))

        # Append the posterior probability to the list
        posteriors.append(posterior)

    # Choose the class label with the highest posterior probability as the prediction
    prediction = np.argmax(posteriors)

    # Return a tuple of true label and predicted label
    return (float(y), float(prediction))

In [23]:
# Add a special key-value pair to store the total number of records in the train RDD
frequency_table[(-1, -1)] = train.count()
# Add another special key-value pair to store the number of records with each class label in the train RDD
frequency_table[(-1, 0)] = train.filter(lambda row: row.state == 0).count()
frequency_table[(-1, 1)] = train.filter(lambda row: row.state == 1).count()

bcast = spark.sparkContext.broadcast(frequency_table)

# Apply the predict function on the test RDD using map and pass the frequency_table as an argument using broadcast
predictions = test.map(lambda line: predict(line, bcast))
# Evaluate the accuracy of the predictions using MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="true_label", predictionCol="predicted_label", metricName="accuracy")
# Define the schema of the predictions DataFrame
schema = StructType([
  StructField("true_label", DoubleType(), True),
  StructField("predicted_label", DoubleType(), True)
])
# Create a DataFrame from the predictions RDD
predictions = predictions.toDF(schema=schema)

23/05/08 14:51:07 WARN TaskSetManager: Stage 7 contains a task of very large size (3734 KiB). The maximum recommended task size is 1000 KiB.
23/05/08 14:51:09 WARN TaskSetManager: Stage 8 contains a task of very large size (3734 KiB). The maximum recommended task size is 1000 KiB.
23/05/08 14:51:10 WARN TaskSetManager: Stage 9 contains a task of very large size (3734 KiB). The maximum recommended task size is 1000 KiB.


In [24]:
accuracy = evaluator.evaluate(predictions)
print(f"Test set accuracy = {accuracy}")

23/05/08 14:51:12 WARN TaskSetManager: Stage 10 contains a task of very large size (3734 KiB). The maximum recommended task size is 1000 KiB.


Test set accuracy = 0.7859765014873106
